In [50]:
import pandas as pd
import pyecharts.options as opts
from pyecharts.charts import Line, Tab, Grid
import akshare as ak



# Code

In [51]:
# 宏观数据展示列表
macro_charts = {}

##### 社会融资规模

In [52]:
macro_china_shrzgm_df = ak.macro_china_shrzgm()
macro_china_shrzgm_df['年度'] = macro_china_shrzgm_df['月份'].str[:4]

line_shrzgm = (
    Line(init_opts=opts.InitOpts(
        width='1200px', 
        height='600px', 
        )
    )
    .add_xaxis(xaxis_data=['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec'])
)



for y in macro_china_shrzgm_df.groupby('年度'):
    line_shrzgm.add_yaxis(y[0],y[1]['社会融资规模增量'].to_list())

line_shrzgm.set_global_opts(
    title_opts=opts.TitleOpts(title="历年社会融资规模增量"),
    legend_opts=opts.LegendOpts(is_show=True, textstyle_opts=opts.TextStyleOpts(color='white')),
    tooltip_opts=opts.TooltipOpts(
        is_show=True,
        trigger_on="mousemove|click",
        axis_pointer_type='cross'),
    xaxis_opts=opts.AxisOpts(
        type_="category",
        splitline_opts=opts.SplitLineOpts(is_show=False, linestyle_opts=opts.LineStyleOpts(type_='dotted'))
    ),
    yaxis_opts=opts.AxisOpts(
        type_="value",
        splitline_opts=opts.SplitLineOpts(is_show=False, linestyle_opts=opts.LineStyleOpts(type_='dotted')),
    ),
)
line_shrzgm.set_series_opts(label_opts=opts.LabelOpts(is_show=False))


macro_charts['社会融资规模增量'] = line_shrzgm

##### PMI

In [53]:
# 财新制造业PMI https://akshare.akfamily.xyz/data/index/index.html#id54
pmi_cx_df = ak.index_pmi_man_cx()

# 股票指数的历史数据 https://akshare.akfamily.xyz/data/index/index.html#id4
stock_zh_index_daily_df = ak.stock_zh_index_daily(symbol="sh000001") # 'sh000001' 上证指数
stock_zh_index_daily_df['date'] = pd.to_datetime(stock_zh_index_daily_df['date'])

# 日频数据重采样，转为月频数据
stock_zh_index_monthly_df = stock_zh_index_daily_df.resample('M', on='date').agg({
    'open':'first',
    'high':'max',
    'low':'min',
    'close':'last',
    'volume':'sum'
})


line_pmi = (
    Line(init_opts=opts.InitOpts(
        width='1200px', 
        height='600px', 
        theme='white',
        # bg_color='rgba(123, 200, 88, 0.4)'
        )
    )
    .add_xaxis(xaxis_data=pd.to_datetime(pmi_cx_df['日期']).dt.strftime("%Y-%m"))

    .add_yaxis(
        series_name="制造业PMI",
        y_axis=pmi_cx_df['制造业PMI'].values.tolist(),
        label_opts=opts.LabelOpts(is_show=False),
        yaxis_index=0
    )

    .add_yaxis(
        series_name="上证指数",
        y_axis=stock_zh_index_monthly_df.loc[pmi_cx_df['日期'].iloc[0]:][:len(pmi_cx_df)]['close'].to_list(),
        label_opts=opts.LabelOpts(is_show=False),
        yaxis_index=1
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title_textstyle_opts=opts.TextStyleOpts(color='white'),
            subtitle_textstyle_opts=opts.TextStyleOpts(color='gray')
        ),
        legend_opts=opts.LegendOpts(is_show=True, textstyle_opts=opts.TextStyleOpts(color='white')),
        tooltip_opts=opts.TooltipOpts(
            is_show=True,
            trigger_on="mousemove|click",
            axis_pointer_type='cross'),
        xaxis_opts=opts.AxisOpts(
            type_="category",
            splitline_opts=opts.SplitLineOpts(is_show=False, linestyle_opts=opts.LineStyleOpts(type_='dotted'))
        ),
        yaxis_opts=opts.AxisOpts(
            type_="value",
            min_ = 20,
            max_ = 60,
            splitline_opts=opts.SplitLineOpts(is_show=False, linestyle_opts=opts.LineStyleOpts(type_='dotted')),
        ),
        datazoom_opts=opts.DataZoomOpts(range_start=3000, range_end=100),
    )
    
    .set_series_opts(
        # 为了不影响标记点，这里把标签关掉
        label_opts=opts.LabelOpts(is_show=False),
        markline_opts=opts.MarkLineOpts(
            data=[
                opts.MarkLineItem(y=50, name="荣枯线"),
                # opts.MarkLineItem(type_="average", name="平均值"),
            ]
        ),
    )

    .extend_axis(
        yaxis=opts.AxisOpts(
            type_="value",
            min_=2000,  # 最小值
            max_=5000,  # 最大值
            splitline_opts=opts.SplitLineOpts(is_show=False, linestyle_opts=opts.LineStyleOpts(type_='dotted'))
        )
    )

)

macro_charts['制造业PMI'] = line_pmi

##### GDP

In [57]:
macro_china_gdp_yearly_df = ak.macro_china_gdp_yearly()
# 先将macro_china_gdp_yearly_df['date']往后推一个月，然后判断属于第几个季度
macro_china_gdp_yearly_df['date_offset'] = macro_china_gdp_yearly_df['date'] - pd.DateOffset(months=1)
macro_china_gdp_yearly_df['year'] = macro_china_gdp_yearly_df['date_offset'].dt.year.astype(str)
macro_china_gdp_yearly_df['quarter'] = macro_china_gdp_yearly_df['date_offset'].dt.quarter.astype(str)
gdp_quarter_list = macro_china_gdp_yearly_df.groupby(['year','quarter']).mean(numeric_only=True).values.tolist()[1:]
gdp_year_list = macro_china_gdp_yearly_df.groupby(['year']).mean(numeric_only=True).values.tolist()[1:]

line_gdp_year_chart = (
    Line()
    .add_xaxis(xaxis_data=macro_china_gdp_yearly_df['year'].unique().tolist()[1:])
    .add_yaxis(
        series_name="年度GDP",
        y_axis=gdp_year_list,

    )
)

line_gdp_quarter_chart = (
    Line()
    .add_xaxis(xaxis_data=(macro_china_gdp_yearly_df['year'] + '-' + macro_china_gdp_yearly_df['quarter']).to_list()[1:])
    .add_yaxis(
        series_name="季度GDP",
        y_axis=gdp_quarter_list,
    )
    .set_series_opts(
        # 为了不影响标记点，这里把标签关掉
        label_opts=opts.LabelOpts(is_show=False),

    )
)


grid = Grid()
grid.add(line_gdp_year_chart, grid_opts=opts.GridOpts(pos_top="10%", pos_bottom="50%"))
grid.add(line_gdp_quarter_chart, grid_opts=opts.GridOpts(pos_top="60%", pos_bottom="10%"))


macro_charts['GDP'] = grid




# 图表

In [58]:
tabs = Tab()
# 遍历字典，添加到tabs中
for k, v in macro_charts.items():
    tabs.add(v, k)
tabs.render_notebook()